<a href="https://colab.research.google.com/github/hobezhang/NLP-based-information-retrieval-system/blob/main/SPLADE-CLOZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_id = 'naver/splade-cocondenser-ensembledistil'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [2]:
passage = '''
Caravanserais were roadside inns that were built along the Silk Road in areas including China,North Africa and the Middle East.They were typically __1_ outside the walls of a city or village and were usually funded by governments of _2__.

This word“Caravanserais”is a __3__ of the Persian word“karvan”,which means a group of travellers or a caravan,and seray,a palace or enclosed building.The Perm caravan was used to __4__ groups of people who travelled together across the ancient network for safety reasons,__5__ merchants,travellers or pilgrims.

From the 10th century onwards,as merchant and travel routes become more developed,the __6__ of the Caravanserais increased and they served as a safe place for people to rest at night.Travellers on the Silk Road __7__ possibility of being attacked by thieves or being __8__ to extreme conditions.For this reason,Caravanserais were strategically placed __9__ they could be reached in a day’s travel time.

Caravanserais served as an informal __10__ point for the various people who travelled the Silk Road.__11__,those structures became important centers for culture __12__ and interaction,with travelers sharing their cultures,ideas and beliefs,__13__ talking knowledge with them,greatly __14__ the development of several civilizations.

Caravanserais were also an important marketplace for commodities and __15__ in the trade of goods along the Silk Road.__16__,it was frequently the first stop merchants looking to sell their wares and __17__ supplies for their own journeys.It is __18__ that around 120000 to 15000 caravanserais were built along the Silk Road,__19__ only about 3000 are known to remain today,many of which are in __20__.'''

options = '''
1.A.displayed B.occupied C.located D.equipped

2.A.privately B.regularly C.respectively D.permanently

3.A.definition B.transition C.substitution D.combination

4.A.classify B.record C.describe D.connect

5.A.apart from B.instead of C.such as D.along with

6.A.construction B.restoration C.impression D.evaluation

7.A.doubted B.faced C.accepted D.reduced

8.A.assigned B.subjected C.accustomed D.opposed

9.A.so that B.even if C.now that D.in case

10.A.talking B.starting C.breaking D.meeting

11.A.by the way B.on occasion C.in comparison D.As a result

12.A.heritage B.revival C.exchange D.status

13.A.with regard to B.in spite of C.as well as D.in line with

14.A.completing B.influencing C.resuming D.pioneering

15.A.aided B.invested C.failed D.competed

16.A.Rather B.Indeed C.otherwise D.However

17.A.go in for B.standard up for C.lose in on D.stock up on

18.A.believed B.predicted C.recalled D.implied

19.A.until B.because C.unless D.although

20.A.ruins B.debt C.fashion D.series
'''



In [3]:
def separatePassage(passage):
    '''
    用于把文章分成句子,一句以句号结尾,并把每个句子的'\n'去掉

    passage: str
    return: list[str]
    '''
    sentences = []
    sentence = ''
    for char in passage:
        if char == '.':
            sentence += char
            sentences.append(sentence.strip())
            sentence = ''
        else:
            sentence += char
    return sentences

sentences = separatePassage(passage)
sentences

['Caravanserais were roadside inns that were built along the Silk Road in areas including China,North Africa and the Middle East.',
 'They were typically __1_ outside the walls of a city or village and were usually funded by governments of _2__.',
 'This word“Caravanserais”is a __3__ of the Persian word“karvan”,which means a group of travellers or a caravan,and seray,a palace or enclosed building.',
 'The Perm caravan was used to __4__ groups of people who travelled together across the ancient network for safety reasons,__5__ merchants,travellers or pilgrims.',
 'From the 10th century onwards,as merchant and travel routes become more developed,the __6__ of the Caravanserais increased and they served as a safe place for people to rest at night.',
 'Travellers on the Silk Road __7__ possibility of being attacked by thieves or being __8__ to extreme conditions.',
 'For this reason,Caravanserais were strategically placed __9__ they could be reached in a day’s travel time.',
 'Caravanserais

In [4]:
import re
def separateOptions(options):
    '''
    用于句子里的选项分开，用字典的方式存储,其中题号作为键，ABCD作为值

    options: str
    return: dict
    '''
    options_dict = {}
    options = options.strip().split('\n')
    index_pattern = re.compile(r'(\d+)\.')
    select_pattern = re.compile(r'([A-Za-z])\.(\w+)')
    for i, option in enumerate(options):
        if len(option) > 0:
            index = index_pattern.findall(option)
            select = select_pattern.findall(option)
            select_dict_list = [{data[0]:data[1]} for data in select]
            select_dict = {}
            for data in select_dict_list:
                select_dict.update(data)

            options_dict[index[0]] = select_dict
    return options_dict

options_dict = separateOptions(options)
options_dict


{'1': {'A': 'displayed', 'B': 'occupied', 'C': 'located', 'D': 'equipped'},
 '2': {'A': 'privately',
  'B': 'regularly',
  'C': 'respectively',
  'D': 'permanently'},
 '3': {'A': 'definition',
  'B': 'transition',
  'C': 'substitution',
  'D': 'combination'},
 '4': {'A': 'classify', 'B': 'record', 'C': 'describe', 'D': 'connect'},
 '5': {'A': 'apart', 'B': 'instead', 'C': 'such', 'D': 'along'},
 '6': {'A': 'construction',
  'B': 'restoration',
  'C': 'impression',
  'D': 'evaluation'},
 '7': {'A': 'doubted', 'B': 'faced', 'C': 'accepted', 'D': 'reduced'},
 '8': {'A': 'assigned', 'B': 'subjected', 'C': 'accustomed', 'D': 'opposed'},
 '9': {'A': 'so', 'B': 'even', 'C': 'now', 'D': 'in'},
 '10': {'A': 'talking', 'B': 'starting', 'C': 'breaking', 'D': 'meeting'},
 '11': {'A': 'by', 'B': 'on', 'C': 'in', 'D': 'As'},
 '12': {'A': 'heritage', 'B': 'revival', 'C': 'exchange', 'D': 'status'},
 '13': {'A': 'with', 'B': 'in', 'C': 'as', 'D': 'in'},
 '14': {'A': 'completing',
  'B': 'influencing',

In [5]:
import itertools
import torch
import numpy as np

def analyze(index, sentences, options_dict, model,tokenizer):
    sentence = sentences[index].split(' ')
    items = []
    candidates = []
    for i, word in enumerate(sentence):
        pattern = re.compile(r'_(\d+)_')
        if len(pattern.findall(word)) > 0:
            items.append(*pattern.findall(word))
            sentence[i] = "[CLOZE]"

    options = [options_dict[item] for item in items]
    # 获取[CLOZE]的位置
    cloze_indices = [i for i, word in enumerate(sentence) if word == "[CLOZE]"]
    # 生成所有可能的组合
    combinations = list(itertools.product(*[option.values() for option in options]))
    for combo in combinations:
        new_sentence = sentence
        for i, cloze_index in enumerate(cloze_indices):
            new_sentence[cloze_index] = combo[i]

        candidates.append(' '.join(new_sentence))

    candidates_len = len(candidates)
    if index - 1 >= 0:
      candidates.append(sentences[index-1])
    else:
      candidates.append(sentences[index+1])

    tokens = tokenizer(
    candidates, return_tensors='pt',
    padding=True, truncation=True
).to(model.device)
    output = model(**tokens)


    vecs = torch.max(
        torch.log(1 + torch.relu(output.logits)) * tokens.attention_mask.unsqueeze(-1), dim=1
    ).values.squeeze().detach().cpu().numpy()
    sim = np.zeros((vecs.shape[0], vecs.shape[0]))

    for i, vec in enumerate(vecs):
        sim[i,:] = np.dot(vec, vecs.T) / (
            np.linalg.norm(vec) * np.linalg.norm(vecs, axis=1)
        )
    sim = sim[candidates_len]
    max_index = np.argmax(sim[:-1])
    print(candidates[max_index])




sentences = separatePassage(passage)
# 找到句子里包含匹配"_数字_"的句子
pattern = re.compile(r'_\d+_')

options_dict = separateOptions(options)
for i, sentence in enumerate(sentences):

    if len(pattern.findall(sentence)) > 0:
        analyze(i, sentences, options_dict,model,tokenizer)

    else:
      print(sentence)

Caravanserais were roadside inns that were built along the Silk Road in areas including China,North Africa and the Middle East.
They were typically located outside the walls of a city or village and were usually funded by governments of regularly
This word“Caravanserais”is a combination of the Persian word“karvan”,which means a group of travellers or a caravan,and seray,a palace or enclosed building.
The Perm caravan was used to describe groups of people who travelled together across the ancient network for safety instead merchants,travellers or pilgrims.
From the 10th century onwards,as merchant and travel routes become more developed,the impression of the Caravanserais increased and they served as a safe place for people to rest at night.
Travellers on the Silk Road reduced possibility of being attacked by thieves or being accustomed to extreme conditions.
For this reason,Caravanserais were strategically placed now they could be reached in a day’s travel time.
Caravanserais served as